In [19]:
import numpy as np
import sys
sys.path.insert(0,'../model')
import mega

In [20]:
hip = np.genfromtxt('../data/hipparcos-bright-result.csv', delimiter=',',skip_header=1,usecols=[1])
visor,magC,adu,npix,bkgnd,streak,rate = np.genfromtxt("../data/mags-streaks.dat",usecols=(1,2,3,4,5,6,7),unpack=True)
gmag = -2.5*np.log10(adu - npix*bkgnd) + magC -2.5 * np.log10(30* rate/streak)

In [21]:
sims = mega.get_simulations()

In [22]:
print("\\multicolumn{3}{c}{\\textbf{Background stars}}& ",end='')
for l, maglim in enumerate([5.,6.5,7.]):                    
    print("\\textbf{%d} & "%len(hip[hip<maglim]), end='')
    if l!=2:
        print("& ", end='')
#print("\\\\ \hline")

for i, latitude in enumerate([60,50,40,30,20]):
    for j, month in enumerate([3,0]):
        hours = [0]
        lon = mega.length_of_night(month=month, latitude=latitude, p=12)
        if lon>0:
            hours.append(lon/2.)
        for k, hour in enumerate(hours):
            if j==0 and k==0:
                print("%2.0f$^{\circ}$N/S & "%latitude, end='')
            else:
                print("                & ",end='')

            if k==0:
                if month==3:
                    print("summer  & ",end='')
                elif month==0:
                    print("equinox & ",end='')
                else:
                    print("        &",end='')
            else:
                print("        & ",end="")

            if hour==0:
                print("midnight           & ", end='')
            else:
                print("nautical dusk/dawn & ", end='')    
            for l, maglim in enumerate([5.,6.5,7.]):    
                vissats = 0
                oversample = 10
                for m in range(oversample):                    
                    xy, mag = mega.get_stereographic_data(sims,latitude=latitude, month=month, hour=hour, airmassCoeff=0.0, randomCoeff=0.2)
                    vissats += len(mag[mag<maglim])
                vissats /= oversample
                print("%3.0f & "%vissats, end='')
                print("%.0f\%% "% (vissats/(vissats+len(hip[hip<maglim])/2.)*100.), end='')
                if l!=2:
                    print("& ",end='')
                else:
                    print("\\\\ ",end='')
            if k==1 and j==1:
                print("\\hline", end='')
            print("")


\multicolumn{3}{c}{\textbf{Background stars}}& \textbf{1608} & & \textbf{8789} & & \textbf{15404} & 60$^{\circ}$N/S & summer  & midnight           &  32 & 4\% & 407 & 8\% & 621 & 7\% \\ 
                & equinox & midnight           &   0 & 0\% &  14 & 0\% &  26 & 0\% \\ 
                &         & nautical dusk/dawn &  14 & 2\% & 214 & 5\% & 359 & 4\% \\ \hline
50$^{\circ}$N/S & summer  & midnight           &  72 & 8\% & 321 & 7\% & 497 & 6\% \\ 
                &         & nautical dusk/dawn & 105 & 12\% & 440 & 9\% & 663 & 8\% \\ 
                & equinox & midnight           &   0 & 0\% &   0 & 0\% &   0 & 0\% \\ 
                &         & nautical dusk/dawn & 130 & 14\% & 395 & 8\% & 562 & 7\% \\ \hline
40$^{\circ}$N/S & summer  & midnight           &   0 & 0\% &  58 & 1\% & 112 & 1\% \\ 
                &         & nautical dusk/dawn & 125 & 13\% & 435 & 9\% & 645 & 8\% \\ 
                & equinox & midnight           &   0 & 0\% &   0 & 0\% &   0 & 0\% \\ 
               

In [23]:
#all sky flux/sq. degree
sum(100**((-hip)/5.)*3631.)/20626.5

26.55461426681228

In [24]:
#all sky flux/sq. degree for satellites
xy, mag = mega.get_stereographic_data(sims, latitude=50, month=3, hour=-3)
sum(100**((-mag)/5.)*3631.)/20626.5

0.5635949244521643